In [13]:
# ==============================================================================
# PORTFOLIO PROJECT: AML ANOMALY DETECTION PROTOTYPE
# BY: Theodorus
# VERSION: 2.1 (Error fix in report generation)
# ==============================================================================

In [14]:
# Please run 'pip install pandas numpy' in your terminal if you haven't already
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os